In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from keras.applications.vgg16 import VGG16
#from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [2]:
# Setting. Can change.
CLASSES = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41']

batch_size_init = 24
batch_size_finetuning = 12

lr_init = 1e-4
lr_finetuning = 1e-6

epochs_init = 10
epochs_finetuning = 30
epochs_all = epochs_init + epochs_finetuning

#input image width, height and shape
IMG_HEIGHT = 200
IMG_WIDTH = 200
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)


#define path
dataset_dir = 'data/dataset'
trained_model_dir = 'trained_model'
results_dir = 'results'

dir_name = 'VGG'

model_checkpoint_dir = f'{trained_model_dir}/{dir_name}'
if not os.path.exists(model_checkpoint_dir):
    os.makedirs(model_checkpoint_dir)
    
model_init = os.path.join(model_checkpoint_dir, 'model_pre_final.h5')
    
chart_stats_dir = f'{results_dir}/{dir_name}'
if not os.path.exists(chart_stats_dir):
    os.makedirs(chart_stats_dir)

train_dir = f'{dataset_dir}/train/train/'
test_dir = f'{dataset_dir}/test/'



In [3]:
def get_generator(batch_size, is_augment=False):
    if is_augment:
        datagen = ImageDataGenerator(
            preprocessing_function=preprocess_input,
            brightness_range=(0.4,0.9),
            horizontal_flip=True,
            validation_split=0.2
        )
    else:
        datagen = ImageDataGenerator(
            preprocessing_function=preprocess_input,
            validation_split=0.2
        )

    
    train_generator = datagen.flow_from_directory(
        train_dir, 
        subset='training',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=batch_size,
        classes=CLASSES,
        class_mode="categorical",
        shuffle=False
    )

    val_generator = datagen.flow_from_directory(
        train_dir,
        subset='validation',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=batch_size,
        classes=CLASSES,
        class_mode="categorical",
        shuffle=False
    )
    
    return train_generator, val_generator

In [4]:
def get_weight(model_path):
    loaded_model = load_model(model_path)
    return loaded_model.get_weights()


def create_model(lr, weight=None):
    base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE,
        classes=42,)
    
    if not weight:
        for layer in base_model.layers:
            layer.trainable = False
            
    x = base_model.layers[-1].output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(rate=0.3)(x)
    predictions = Dense(len(CLASSES), activation='softmax')(x)
    model = Model(inputs=base_model.inputs, outputs=predictions)
    
    if weight:
        model.set_weights(weight)

    # compile model
    opt = Adam(lr=lr)
    model.compile(
        optimizer=opt,
        loss=categorical_crossentropy,
        metrics=['accuracy']
    )
    
    return model

In [5]:
train_generator, validation_generator = get_generator(batch_size_init)

Found 84331 images belonging to 42 classes.
Found 21061 images belonging to 42 classes.


In [5]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = create_model(lr_init)


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 99, 99, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 99, 99, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 99, 99, 32)   0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [8]:
hist_init = model.fit(
    x=train_generator,
    epochs=epochs_init,
    validation_data=validation_generator
    
)
model.save(model_init)

Train for 3514 steps, validate for 878 steps
Epoch 1/10
3514/3514 [==============================] - 2963s 843ms/step - loss: 3.8441 - accuracy: 0.0245 - val_loss: 3.7539 - val_accuracy: 0.0452
Epoch 2/10
3514/3514 [==============================] - 2341s 666ms/step - loss: 3.8208 - accuracy: 0.0270 - val_loss: 3.6611 - val_accuracy: 0.0556
Epoch 3/10
3514/3514 [==============================] - 2358s 671ms/step - loss: 3.8116 - accuracy: 0.0287 - val_loss: 3.5867 - val_accuracy: 0.0754
Epoch 4/10
3514/3514 [==============================] - 2347s 668ms/step - loss: 3.8118 - accuracy: 0.0288 - val_loss: 3.4707 - val_accuracy: 0.1303
Epoch 5/10
3514/3514 [==============================] - 2351s 669ms/step - loss: 3.8066 - accuracy: 0.0305 - val_loss: 3.4203 - val_accuracy: 0.1568
Epoch 6/10
3514/3514 [==============================] - 2355s 670ms/step - loss: 3.7995 - accuracy: 0.0323 - val_loss: 3.3787 - val_accuracy: 0.1763
Epoch 7/10
3514/3514 [==============================] - 2360s

OSError: Unable to create file (unable to open file: name = 'trained_model/VGG\model_pre_final.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [10]:
weight = get_weight(model_init)

model = create_model(lr_finetuning, weight)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 99, 99, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 99, 99, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 99, 99, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [ ]:
checkpoint = ModelCheckpoint(
        filepath=model_checkpoint_dir+'/model_weights_{epoch}.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1,
        save_freq='epoch'
)

train_generator, validation_generator = get_generator(batch_size_finetuning, True)

hist_fine = model.fit(
    x=train_generator,
    epochs=1,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

Found 84331 images belonging to 42 classes.
Found 21061 images belonging to 42 classes.


In [21]:
checkpoint = ModelCheckpoint(
        filepath=model_checkpoint_dir+'/model_weights_{epoch}.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1,
        save_freq='epoch'
)

In [15]:
def show_acc_loss_graph(hist_init, hist):
    acc = hist_init.history['accuracy'] + hist.history['accuracy']
    val_acc = hist_init.history['val_accuracy'] + hist.history['val_accuracy']
    loss = hist_init.history['loss'] + hist.history['loss']
    val_loss = hist_init.history['val_loss'] + hist.history['val_loss']
    
    # save accuracy graph image
    plt.plot(range(1, epochs_all+1), acc, marker='.', label='accuracy')
    plt.plot(range(1, epochs_all+1), val_acc, marker='.', label='val_accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.ylim(top=1.0) 
    plt.savefig(os.path.join(chart_stats_dir, 'accuracy.png'))
    plt.show()

    # save loss graph image
    plt.plot(range(1, epochs_all+1), loss, marker='.', label='loss')
    plt.plot(range(1, epochs_all+1), val_loss, marker='.', label='val_loss')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.ylim(bottom=0.0) 
    plt.savefig(os.path.join(chart_stats_dir, 'loss.png'))
    plt.show()

show_acc_loss_graph(hist_init, hist_fine)


AttributeError: 'ModelCheckpoint' object has no attribute 'history'

Run on Test Set to get submission csv file

In [15]:
#load the trained model back
trained_model = create_model(lr, model_weight_path)

NameError: name 'lr' is not defined

In [3]:
#load the trained model back
model = load_model(model_checkpoint_dir+'/model_weights_7.h5')
model.trainable = False

In [4]:
# compile model
opt = Adam(lr=lr_finetuning)
model.compile(optimizer=opt,loss=categorical_crossentropy, metrics=['accuracy'])


In [5]:
import pandas as pd
result_df = pd.DataFrame(columns=['filename', 'category'])
result_df

,filename,category


In [6]:
#get the list of images for prediction
img_files = [file for file in os.listdir(test_dir)]

#preprocess the images and get predictions
for img_file in img_files:
    input_img = load_img(f'{test_dir}/{img_file}', target_size=(IMG_WIDTH, IMG_HEIGHT))
    img = img_to_array(input_img)
    img = img.reshape(1,IMG_WIDTH,IMG_HEIGHT,3)
    img = preprocess_input(img)
    prediction = model.predict(img)
    
    predicted_class = CLASSES[prediction.argmax()]
    
    result_df = result_df.append({'filename':img_file, 'category': predicted_class}, ignore_index=True)

In [7]:
result_df

,filename,category
0,0004b03ad7eabfb3989727c461310a84.jpg,12
1,0004d34271d0a0253f8a95d0746bae05.jpg,16
2,000d4ad3d0a372fd6eea2fee6bf738ef.jpg,20
3,0012f125ad07123e69468ae1d1d59de4.jpg,22
4,0015831b249b856e569e883a9dc6d4fa.jpg,12
5,001640e7947935ad28d00f740c3eaefc.jpg,00
6,00196f6ef552b2c4d78365818acdbce2.jpg,40
7,001d40e7718c79ed31750d6c988a90fc.jpg,30
8,001eeb075533bafbfb8e4a6f9e4b8765.jpg,10
9,0022bd7f6b85fe98edd3291d9824ee6c.jpg,41


In [8]:
result_df.to_csv(chart_stats_dir+'/submission.csv', index=False)